In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [14]:
df = pd.read_csv('data/data.csv')

In [15]:
df.head()

,Text,Polarity
0,আমাদের সজাগ থাকতে হবে টিকা নেওয়া নিয়ে।,1
1,ভ্যাকসিন ভালো না।,0
2,আমার পরিবারের সকলেই টিকা নিয়েছে।,1
3,টিকা লক্ষণীয় রোগ প্রতিরোধে একইভাবে উচ্চ কার্য...,1
4,অনলাইনে আবেদন করে আমি ভ্যাকসিন নিয়েছি।,1


In [16]:
df_shuffle = df.sample(frac = 1,random_state=42)
df_shuffle.head()

,Text,Polarity
1011,কেন্দ্রে প্রচণ্ড ভিড় থাকায় অনেকে ভেক্সিন পায...,0
3182,ভ্যাকসিন দিতে গিয়ে কোনরকম ঝামেলার সম্মুখীন হত...,1
3350,করোনার ভ্যাকসিনগুলি আপনাকে করোনায় অসুস্থ করতে ...,1
897,ভ্যাকসিন নিয়ে আমি ভালো আছি।,1
2293,টিকা ছাড়াও ভালো হয়।,0


In [17]:
df['Polarity'].value_counts()

1    2007
0    1800
Name: Polarity, dtype: int64

In [18]:
def save_model(model,filename):
    with open(f'models/{filename}.pkl','wb') as f:
        pickle.dump(model,f)

In [19]:
train_sentence,test_sentence,train_label,test_label = train_test_split(df_shuffle['Text'].to_numpy(),df_shuffle['Polarity'].to_numpy(),test_size=0.2,random_state=42)

In [20]:
train_sentence[:10]

array(['সময় মত ২য় ডোজ নিতে দেরি করলে মানুষ মারা যায়।',
       'টিকা নিতে হবে। টিকে থাকতে হবে এই মহামারীতে।',
       'টিকা নেওয়ার পর আমার জ্বর এসেছিলো। আমি মনে করি যে টিকাটি আমার শরীরের ঠিক কাজ করেছে।',
       'কিছু ধরণের COVID-19 ভ্যাকসিন মেসেঞ্জার RNA (mRNA) ব্যবহার করে তৈরি করা হয়েছে, এটি একটি নতুন প্রযুক্তি যা প্রচলিত পদ্ধতির ভ্যাকসিন তৈরির চেয়ে দ্রুত পদ্ধতির অনুমতি দেয়।',
       'আপনার কাছে যখন বিকল্প উপায়ে থাকে, তখন ভ্যাকসিন নেওয়ার কোনো প্রয়োজন নেই।',
       'ভেকসিনই একমাত্র উপায় নয় আরও অনেক উপায় আছে।',
       'সরকারের পক্ষ থেকে ইউনিয়নভিত্তিক টিকা প্রদানের যে পরিকল্পনা গ্রহণ করা হয়েছে সেখানে অনেক দুর্নীতি হচ্ছে। ',
       'টিকার জন্য নিবন্ধন করেছি কিন্তু আমার এসএমএস আসছে না',
       'আমি বিদেশ যাবো দেখে আগে আগেই টিকা নিয়ে নিছি। ',
       'টিকা গ্রহণের পর দুর্বল হয়ে যায় বা টিকা গ্রহণের পর লোকে মারা যায়, গ্রাম ও প্রত্যন্ত অঞ্চলে এই ধরণের লোকেদের ভুল বিশ্বাস ভাঙা উচিত বলেও জানিয়েছেন।'],
      dtype=object)

In [21]:
# visualize some random training examples

import random

random_index = random.randint(0,len(train_sentence) -5)

for row in df_shuffle[['Text','Polarity']][random_index:random_index+5].itertuples():
    _,text,polarity = row
    print(f"Target:{polarity}", "(Positive FeedBack)" if polarity > 0 else "(Negative FeedBack)")
    print(f"Text\n {text}\n")
    print("---\n")


Target:1 (Positive FeedBack)
Text
 করোনা মহামারি প্রতিরোধক ভ্যাকসিন ব্যবহারের ফলে ধনী দেশগুলোর রাষ্ট্রনেতারা যথেষ্ট সচল।

---

Target:0 (Negative FeedBack)
Text
 টিকা নিলেই কি সব ঠিক হয়ে যাবে নাকি ভাই।

---

Target:1 (Positive FeedBack)
Text
 বিভিন্ন পেশায় চাকরি করতে হলে তাদের টিকা নেওয়া বাধ্যতামূলক করা হয়েছে।

---

Target:0 (Negative FeedBack)
Text
 আপানার কি বাঁচার ইচ্ছা নাই যে ভেকসিন নিতে যান।

---

Target:0 (Negative FeedBack)
Text
 ভ্যাকসিন নিয়ে করোনা কে জয় করা যায় না। 

---



In [22]:

len(train_sentence), len(train_label),len(test_sentence),len(test_label)

(3045, 3045, 762, 762)

In [23]:
import tensorflow as tf
from keras.layers.preprocessing.text_vectorization import TextVectorization
import numpy as np

text_vec = TextVectorization()

In [24]:
average_word_per_sentence = round(sum(len(i.split()) for i in train_sentence)/len(train_sentence))

### Settings Parmaeters for TextVectorization




In [25]:
max_vocab_length = 10000
max_length = average_word_per_sentence
text_vectorized = TextVectorization(
    max_tokens=max_vocab_length,
    output_mode='int',
    output_sequence_length=max_length)

In [26]:
text_vectorized.adapt(train_sentence)

In [27]:
sample_sentence = 'সময় মত ২য় ডোজ নিতে দেরি করলে মানুষ মারা যায়।'

text_vectorized([sample_sentence])

<tf.Tensor: shape=(1, 11), dtype=int64, numpy=
array([[152, 498, 522,  10,  14, 672, 569,  33, 175, 189,   0]],
      dtype=int64)>

In [28]:

random_sentence = random.choice(train_sentence)
print(f"Original text:\n\n{random_sentence}\n\n Vectorized version : {text_vectorized([random_sentence])}")

Original text:

টিকার সুফল ও করোনা থেকে রক্ষা পেতে হলে সর্বস্তরের জন সাধারনের টিকা নিশ্চিত করতে হবে। তা না হলে করোনার ভয়াল থাবা থেকে রক্ষা পাওয়া সম্ভব নয়।

 Vectorized version : [[  21  903   23    7   13  138  741  103 2908 2373 2886]]


In [29]:


word_to_vocab = text_vectorized.get_vocabulary()

top_10_word = word_to_vocab[:10]
bottom_10_word = word_to_vocab[-10:]

print(f"Number of words in vocabulary {len(word_to_vocab)}\n")
print(f"Top 10 words : {top_10_word}")
print(f"Bottom 10 words : {bottom_10_word}")

Number of words in vocabulary 5201

Top 10 words : ['', '[UNK]', 'টিকা', 'ভ্যাকসিন', 'আমি', 'না।', 'না', 'করোনা', 'জন্য', 'এবং']
Bottom 10 words : ['c130j', 'astrazenecaএর', '95।', '95', '90', '6', '3', '25', '18', '10']


In [30]:
# making an embedding layer

from keras.layers import Embedding

embedding = Embedding(
    input_dim=max_vocab_length,
    output_dim=128,
    input_length=max_length,
   
)
embedding

In [31]:
random_sentence = random.choice(train_sentence)

print(f" Original Text :\n\n {random_sentence} \n\n Embedding Version : ")
sample_emb = embedding(text_vectorized([random_sentence])) 
sample_emb

 Original Text :

 করোনার টিকা আর বাকি টিকাদের মতই একটি টিকা। 

 Embedding Version : 


<tf.Tensor: shape=(1, 11, 128), dtype=float32, numpy=
array([[[ 0.00623472,  0.01150583, -0.01068955, ...,  0.00288393,
          0.00507841, -0.02556775],
        [-0.04921652,  0.00369044, -0.03969527, ..., -0.01253337,
         -0.00093545, -0.00275923],
        [ 0.03785666,  0.04629939,  0.02322091, ...,  0.01282587,
         -0.02592528,  0.03131017],
        ...,
        [ 0.02894287, -0.00671619,  0.04286568, ...,  0.04278977,
          0.03727189, -0.03300672],
        [ 0.02894287, -0.00671619,  0.04286568, ...,  0.04278977,
          0.03727189, -0.03300672],
        [ 0.02894287, -0.00671619,  0.04286568, ...,  0.04278977,
          0.03727189, -0.03300672]]], dtype=float32)>

In [32]:
sample_emb[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([ 0.00623472,  0.01150583, -0.01068955,  0.0070736 ,  0.02283638,
        0.04429081, -0.00692482, -0.01043485, -0.02273856, -0.03094897,
       -0.02109928,  0.04274793, -0.04709503, -0.03268763,  0.01286194,
       -0.04132631,  0.04931352,  0.01187614, -0.0310769 , -0.03641319,
       -0.04286672,  0.03856272, -0.02751043,  0.02480911,  0.00731537,
        0.01447811,  0.04042763, -0.01651819,  0.02290808,  0.01015395,
       -0.03361458,  0.01652456, -0.02116568,  0.02602934, -0.025481  ,
        0.02994723,  0.02218343, -0.031953  ,  0.00626932,  0.00672521,
       -0.03966808, -0.02457812,  0.01151273, -0.03359361, -0.00436624,
       -0.02624904, -0.02324905, -0.00275624, -0.00442274,  0.03205379,
       -0.00506396, -0.02434853,  0.04855938, -0.01452116,  0.0299703 ,
        0.01661168, -0.02689753, -0.02369739,  0.01164397, -0.03308631,
        0.03859761, -0.01864224,  0.04388572,  0.02840302,  0.00255962,
       -0.017035

### Model 1 : Naive Bayes Classifier

In [33]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline


model_naive_bayes = Pipeline([
    ('tfidf',TfidfVectorizer()),
    ('multiNB',MultinomialNB())
])

model_naive_bayes.fit(train_sentence,train_label)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('multiNB', MultinomialNB())])

In [34]:
score = model_naive_bayes.score(test_sentence,test_label)
print(f'Our Naive Bayes accuracy: {score*100:.2f} %')

Our Naive Bayes accuracy: 66.80 %


In [35]:

# Prediction Test

naive_bayes_pred = model_naive_bayes.predict(test_sentence[:5])

naive_bayes_pred[:5], test_label[:5]

(array([0, 0, 0, 1, 0], dtype=int64), array([0, 0, 1, 0, 0], dtype=int64))

In [36]:
model_naive_bayes.predict([test_sentence[1]]),test_label[1]

(array([0], dtype=int64), 0)

In [37]:
rdint = random.randint(0,abs(len(test_sentence) -5))

for i,word in enumerate(test_sentence[rdint:rdint+5]):
  
    print(f"Text: {word}  pred: {model_naive_bayes.predict([word])[0]} original: {test_label[rdint+i+1]}\n\n")

Text: টিকা নিয়াই তো বাঁইচা আছি।টিকা ছাড়া কি আর বাঁচবার পারতাম।   pred: 0 original: 1


Text: মডার্নার ভ্যাকসিন একজন ব্যক্তির কোভিড-১৯ হওয়ার ঝুঁকি ৯৪% কমিয়ে দিয়েছে।  pred: 1 original: 1


Text: ভ্যাকসিন নিয়ে মানসিকভাবেও ভালো থাকেন।   pred: 1 original: 1


Text: একজন মানুষের সুস্থ ও স্বাভাবিক জীবন কাটানোর জন্য ভ্যাকসিন নেওয়া এই সময় জরুরি।   pred: 1 original: 1


Text: দুই ডোজে দুরকম ভ্যাকসিন নিলে মোকাবিলা সম্ভব ডেল্টা ভ্যারিয়েন্টের সঙ্গে।  pred: 1 original: 0




In [38]:
# Calculate Accuracy

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true,y_pred):
    model_accuracy = accuracy_score(y_true,y_pred) *100
    precision,recall,f1,_ = precision_recall_fscore_support(y_true,y_pred,average='weighted')
    model_result = {
        'accuracy':model_accuracy,
        'precision':precision,
        'recall':recall,
        'f1':f1
    }
    return model_result

In [39]:
calculate_results(model_naive_bayes.predict(test_sentence),test_label)

{'accuracy': 66.7979002624672,
 'precision': 0.6960798636606401,
 'recall': 0.6679790026246719,
 'f1': 0.6729967926241313}

In [40]:
import pickle
with open ('models/naive_bayes.pkl', 'wb') as f:
    pickle.dump(model_naive_bayes,f)

In [41]:
def build_model( algorithm, train_sentence=train_sentence, train_label=train_label):
    model_pipeline = Pipeline([
    ('tfidf',TfidfVectorizer()),
    ('Algorithum',algorithm)
    ])
    model_pipeline.fit(train_sentence,train_label)
    return model_pipeline
    

### Logistic   Regression

In [42]:
from sklearn.linear_model import LogisticRegression
model_logistic_regression = build_model(LogisticRegression())
model_logistic_regression.score(test_sentence,test_label)
calculate_results(model_logistic_regression.predict(test_sentence),test_label)

with open ('models/lr.pkl', 'wb') as f:
    pickle.dump(model_naive_bayes,f)

### Build Model with Random Forest, Gradient Boosting, XGBoost, LightGBM

In [43]:
from sklearn.svm import LinearSVC   
model_linear_svc = build_model(LinearSVC())
model_linear_svc.score(test_sentence,test_label)
calculate_results(model_linear_svc.predict(test_sentence),test_label)

{'accuracy': 66.14173228346458,
 'precision': 0.6624484936730389,
 'recall': 0.6614173228346457,
 'f1': 0.6616414616391161}

In [44]:

from sklearn.ensemble import RandomForestClassifier
model_random_forest = build_model(RandomForestClassifier())
model_random_forest.score(test_sentence,test_label)
calculate_results(model_random_forest.predict(test_sentence),test_label)
save_model(model_random_forest,filename='model_random_forest.pkl')

In [45]:

from sklearn.ensemble import GradientBoostingClassifier
model_gradient_boosting = build_model(GradientBoostingClassifier())
model_gradient_boosting.score(test_sentence,test_label)
calculate_results(model_gradient_boosting.predict(test_sentence),test_label)
save_model(model_gradient_boosting,filename='model_gradient_boosting.pkl')

In [46]:

from sklearn.ensemble import AdaBoostClassifier
model_adaboost = build_model(AdaBoostClassifier())
model_adaboost.score(test_sentence,test_label)
calculate_results(model_adaboost.predict(test_sentence),test_label)
save_model(model_adaboost,filename='model_adaboost.pkl')

In [49]:

from sklearn.ensemble import ExtraTreesClassifier
model_extra_tree = build_model(ExtraTreesClassifier())
model_extra_tree.score(test_sentence,test_label)
calculate_results(model_extra_tree.predict(test_sentence),test_label)
save_model(model_extra_tree,filename='model_extra_tree.pkl')

In [50]:

from sklearn.ensemble import BaggingClassifier
model_bagging = build_model(BaggingClassifier())
model_bagging.score(test_sentence,test_label)
calculate_results(model_bagging.predict(test_sentence),test_label)


{'accuracy': 67.97900262467192,
 'precision': 0.6798214546789078,
 'recall': 0.6797900262467191,
 'f1': 0.6798010576232815}

In [51]:

from sklearn.neighbors import KNeighborsClassifier
model_knn = build_model(KNeighborsClassifier())
model_knn.score(test_sentence,test_label)
calculate_results(model_knn.predict(test_sentence),test_label)


{'accuracy': 56.03674540682415,
 'precision': 0.7169998975592242,
 'recall': 0.5603674540682415,
 'f1': 0.5967490612916128}

In [52]:

from sklearn.tree import DecisionTreeClassifier
model_decision_tree = build_model(DecisionTreeClassifier())
model_decision_tree.score(test_sentence,test_label)
calculate_results(model_decision_tree.predict(test_sentence),test_label)


{'accuracy': 63.25459317585301,
 'precision': 0.6331635058759686,
 'recall': 0.6325459317585301,
 'f1': 0.632697917778856}

In [53]:
# make a function to save model

def save_model(model,filename):
    with open(f'models/{filename}.pkl','wb') as f:
        pickle.dump(model,f)

save_model(model_naive_bayes,'naive_bayes')

In [56]:
class BuildModel():

    def __init__(self,train_sentence,train_label,test_sentence,test_label,algo) -> None:
        self.train_sentence = train_sentence
        self.train_label = train_label
        self.test_sentence = test_sentence
        self.test_label = test_label
        self.algo = algo

    
    def run(self):
        self.build_model()
        self.score()
        return self.calculate_results()

    def build_model(self):
        self.model = Pipeline([
            ('tfidf', TfidfVectorizer()),
            ('algo',algo)
        ])
        return self.model.fit(self.train_sentence,train_label)
    
    def score(self):
        return self.model.score(test_sentence,test_label)

    def calculate_results(self):
        y_true = test_label
        y_pred = self.model.predict(test_sentence)
        model_accuracy = accuracy_score(y_true,y_pred) *100
        precision,recall,f1,_ = precision_recall_fscore_support(y_true,y_pred,average='weighted')
        model_result = {
            'accuracy':model_accuracy,
            'precision':precision,
            'recall':recall,
            'f1':f1
        }
        return model_result

        

In [58]:
# Building Neural Network 
import keras as ks
from keras import layers,Model

input = layers.Input(shape=(1,), dtype="string")
x = text_vectorized(input)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
output = layers.Dense(1, activation="sigmoid")(x)
nn_model = Model(input,output)

In [59]:
nn_model.compile(loss='binary_crossentropy',optimizer="Adam",metrics=['accuracy'])

In [60]:
nn_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 11)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 11, 128)           1280000   
                                                                 
 global_average_pooling1d_1   (None, 128)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
Non-tra

In [61]:
nn_history = nn_model.fit(train_sentence,train_label, epochs=5, )

Epoch 1/5
96/96 [==============================] - 3s 19ms/step - loss: 0.6696 - accuracy: 0.7048
Epoch 2/5
96/96 [==============================] - 2s 19ms/step - loss: 0.5634 - accuracy: 0.8141
Epoch 3/5
96/96 [==============================] - 2s 19ms/step - loss: 0.4454 - accuracy: 0.8506
Epoch 4/5
96/96 [==============================] - 2s 18ms/step - loss: 0.3609 - accuracy: 0.8745
Epoch 5/5
96/96 [==============================] - 2s 19ms/step - loss: 0.3003 - accuracy: 0.8972


In [62]:

nn_pred = tf.squeeze(tf.round(nn_model.predict(test_sentence))) 

24/24 [==============================] - 0s 4ms/step


In [63]:
calculate_results(test_label,nn_pred)

{'accuracy': 77.82152230971128,
 'precision': 0.7806410141023981,
 'recall': 0.7782152230971129,
 'f1': 0.7775770282684895}

In [64]:
nn_model.save('models/nn_model')

INFO:tensorflow:Assets written to: models/nn_model\assets


INFO:tensorflow:Assets written to: models/nn_model\assets


In [65]:
save_model(model_logistic_regression,'model_logistic_regration.pkl')

In [ ]:
# save_model(model_random_forest,'model_random_forest.pkl')
save_model(model_linear_svc,'model_linear_svc.pkl')
save_model(model_adaboost,'model_adaboot.pkl')
save_model(model_gradient_boosting,'model_gradient_boosting.pkls')
save_model(model_extra_tree,'model_extra_tree.pkl')


NameError: name 'model_adaboost' is not defined